# Quickstart

This notebook serves as an introduction on getting started with pyCIMS. Before we dive in, make sure you have downloaded or cloned the pyCIMS git repo from https://gitlab.rcg.sfu.ca/mlachain/pycims_prototype/-/tree/master and have ensured that it is up-to-date

Now, we can get started with an example. 

## Import pyCIMS  & other packages
We need to make sure we can access the `pyCIMS` module. Since we are still developing the module, we can't install it from PyPI. Instead, we will make sure the module is in our PATH. The code below assumes you started the jupyter notebook from inside the `pycims_prototype/code/tutorial_examples` directory. 

In [1]:
import os
import sys
import pprint as pp

sys.path.insert(0, os.path.abspath('../../')) # Assumes ipynb ran from pycims_prototype/code/model_validation
sys.path.insert(0, os.path.abspath('../../pyCIMS')) # Assumes ipynb ran from pycims_prototype/code/model_validation

import pyCIMS

We will also need to install the `pyxlsb` package. This is an optional dependency of `pyCIMS` that is required when our model descriptions are saved as `.xlsb` files.   

In [2]:
!pip install pyxlsb

## Model Validator

Now that we have loaded the `pyCIMS` module we can start to use it. First, we will validate our model description using the `ModelValidator` class.

In [3]:
model_description_file = '../../pyCIMS_model_description_Alberta_Validated.xlsb' # Assumes ipynb ran from pycims_prototype/code/tutorial_examples

In [3]:
model_validator = pyCIMS.ModelValidator(model_description_file)
model_validator.validate(verbose=True, raise_warnings=False)

0 node name/branch mismatches. 
0 references to unspecified nodes. 
0 non-root nodes are never referenced. 
0 nodes were specified but don't provide a service. 
0 nodes had invalid competition types. 
2 nodes requested services of themselves. See ModelValidator.warnings['nodes_requesting_self'] for more info
139 nodes or technologies don't request other services. See ModelValidator.warnings['nodes_no_requested_service'] for more info


For more information on using the `ModelValidator` class, checkout the [model validation tutorial](model_validation_instructions.ipynb). 

## Build the Model
There are two steps to building the model. First, we need to create a `ModelReader` object. This object reads in the model description Excel file and does some processessing to prepare the description to become a model. Then, this `ModelReader` object becomes the input to the all important `Model` object.

In [4]:
# Model Reader
model_reader = pyCIMS.ModelReader(infile=model_description_file,
                                  sheet_map={'model': 'Model'},
                                  node_col='Node')

In [5]:
# Model
model = pyCIMS.Model(model_reader)

## Run the pyCIMS Model

In [6]:
model.run(max_iterations=0, show_warnings=False)

***** ***** year: 2000 ***** *****
iter 0


/mnt/c/Users/Steven/Research/CIMS/pycims_prototype/pyCIMS/graph_utils.py:222: UserWarning: Found a Loop -- 
  warnings.warn("Found a Loop -- ")


KeyError: 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'

## Examine the Results

In [7]:
# How has a fuel's price changed between years? 
import pprint as pp

price_of_interest = 'pyCIMS.Canada.Alberta.Electricity'
for year in model.years:
    pp.pprint(model.graph.nodes[price_of_interest][year]['Life Cycle Cost'])


{'Electricity': {'to_estimate': False, 'year_value': 22}}
{'Electricity': {'to_estimate': True, 'year_value': 64.82756490857975}}
{'Electricity': {'to_estimate': True, 'year_value': 62.85170137762043}}
{'Electricity': {'to_estimate': True, 'year_value': 64.62965590274918}}
{'Electricity': {'to_estimate': True, 'year_value': 43.327621701426615}}
{'Electricity': {'to_estimate': True, 'year_value': 48.84997272938841}}
{'Electricity': {'to_estimate': True, 'year_value': 48.278416237267656}}
{'Electricity': {'to_estimate': True, 'year_value': 46.20418892709652}}
{'Electricity': {'to_estimate': True, 'year_value': 43.71927009687332}}
{'Electricity': {'to_estimate': True, 'year_value': 41.755293085468644}}
{'Electricity': {'to_estimate': True, 'year_value': 36.98938603328726}}


In [8]:
# How has marketshare changed between years ?
node_of_interest = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'
for year in model.years:
    print('***** {} *****'.format(year))
    for t, d in model.graph.nodes[node_of_interest][year]['technologies'].items():
        print("{}: {}%".format(t, round(d['total_market_share']*100)))


***** 2000 *****
Incandescent: 92%
CFL: 8%
LED: 0%
***** 2005 *****
Incandescent: 0%
CFL: 100%
LED: 0%
***** 2010 *****
Incandescent: 0%
CFL: 100%
LED: 0%
***** 2015 *****
Incandescent: 0%
CFL: 96%
LED: 0%
***** 2020 *****
Incandescent: 0%
CFL: 99%
LED: 1%
***** 2025 *****
Incandescent: 0%
CFL: 97%
LED: 3%
***** 2030 *****
Incandescent: 0%
CFL: 96%
LED: 4%
***** 2035 *****
Incandescent: 0%
CFL: 95%
LED: 4%
***** 2040 *****
Incandescent: 0%
CFL: 96%
LED: 4%
***** 2045 *****
Incandescent: 1%
CFL: 96%
LED: 4%
***** 2050 *****
Incandescent: 1%
CFL: 96%
LED: 3%


In [9]:
import pickle

In [11]:
with open("model.pkl","wb") as fh:
    pickle.dump(model, fh)

In [13]:
!ls -lh model.pkl

Patching args: ['/bin/bash', '-c', 'ls -lh model.pkl']
Process is not python, returning.
-rwxrwxrwx 1 sbergner sbergner 26M Sep 15 15:56 model.pkl
